In [3]:
#Importing Python Packages
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.ensemble import RandomForestRegressor # type: ignore
from IPython.display import display_html # type: ignore

#Importing All VI CSV's In Chronological Order:

#2020
June18 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2020/June_18_2020_Combined.csv")
June23 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2020/June_23_2020_Combined.csv")
June30 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2020/June_30_2020_Combined.csv")
July6 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2020/July_6_2020_Combined.csv")

#2021
June16 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2021/June_16_2021_Combined.csv")
June22= pd.read_csv("C:/NSF-REU/Height_Combined_Files/2021/June_22_2021_Combined.csv")
June_30 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2021/June_30_2021_Combined.csv")
July12 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2021/July_12_2021_Combined.csv")

#2022
June15 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2022/June_15_2022_Combined.csv")
June_23 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2022/June_23_2022_Combined.csv")
June28 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2022/June_28_2022_Combined.csv")
July13 = pd.read_csv("C:/NSF-REU/Height_Combined_Files/2022/July_13_2022_Combined.csv")
June15 = June15.drop(columns=['id_old'])
June_23 = June_23.drop(columns=['id_old'])
June28 = June28.drop(columns=['id_old'])
July13 = July13.drop(columns=['id_old'])

days = [June18,June23,June30,July6,June16,June22,June_30,July12,June15,June_23,June28,July13] #List of observed dates in chronological order
feature_labels = ['blank'] * 19
importances_list = list()
means = list()
standard_devs = list()
sums = [0.0] * 19
sums_of_features = pd.DataFrame(sums)
blank_index = [''] * 19

#Table caption formatting (for data tables):
def title_case(name):
    year = '\t\t\t\t\t'+ name
    print(year.title()) 

#Data table formatting
def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline; border: 0;"'), 
        raw=True
    )

def feature_importance():
    index = 0
    for features in days: #Iterates through column values on each and every date
        key_features = features.drop(columns=['Plot', 'Year', 'Date', 'Row', 'Range', 'left', 'top', 'right', 'bottom', 'Yld Mass(D', 'Yld Vol(Dr', 'Crop Flw(M', 'Crop Flw(V', 'Mean.Yld.bu.ac', 'X', 'Y'])
        x = key_features
        y = features['Mean.Yld.bu.ac']
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.70)
        model = RandomForestRegressor(n_estimators=100) 
        model.fit(x_train, y_train)
        y_predict = model.predict(x_test)
        
        #Feature Importances
        importances = model.feature_importances_
        
        #Display histograms of feature importance in order by date
        field_importance = pd.Series(importances, index=x.columns)
        field_importance.plot(kind='bar')
        plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
        plt.yticks(np.arange(0, importances.max(), 0.1)) 
        plt.rc('xtick', labelsize=15) 
        plt.rc('ytick', labelsize=25)
        plt.show()

        df = pd.DataFrame(importances)
        for i in range(0,19): 
            sums_of_features.iloc[i] += df.iloc[i]

        # if (len(importances_list) == 0):
        #     daily_imp = pd.DataFrame(importances, index=[blank_index], columns=features['Date'].unique())
        # else:
            daily_imp = pd.DataFrame(importances, index=[blank_index], columns=features['Date'].unique())
        importances_list.append(daily_imp)
        
        #Calculating Means and standard deviations (SDs) for each year, and placing them into lists

        if (len(importances_list) % 4 == 0): # If 4 dates have been analyzed, then calulate Mean & SD for year
            
            means.append(sums_of_features/4) # Means were obtained for each feature from dividing sums_of_features list by 4

            feature_sds = [0.0] * 19
            indiv_features_by_year = [0.0] * 4
            for j in range(0,19):
                for i in range(0,4):
                    indiv_features_by_year[i] = importances_list[i+index].values[j]
                feature_sds[j] = np.std(indiv_features_by_year)
            by_year = pd.DataFrame(feature_sds)
            standard_devs.append(by_year) # SDs
            index +=4 # Increments index so that it will calulate SD for all 4 dates from each year

    #After calculations are made, means in the list are assigned to individual variables
    means_2020 = means[0]
    means_2021 = means[1] - means_2020
    means_2022 = means[2] - means_2021 - means_2020
    
    mean2020 = pd.DataFrame(means_2020.values, index=[blank_index], columns=['Mean'])
    mean2021 = pd.DataFrame(means_2021.values, index=[blank_index], columns=['Mean'])
    mean2022 = pd.DataFrame(means_2022.values, index=[blank_index], columns=['Mean'])

    sdevs_2020 = standard_devs[0]
    sdevs_2021 = standard_devs[1]
    sdevs_2022 = standard_devs[2]

    sdev2020 = pd.DataFrame(sdevs_2020.values, index=[blank_index], columns=['SD'])
    sdev2021 = pd.DataFrame(sdevs_2021.values, index=[blank_index], columns=['SD'])
    sdev2022 = pd.DataFrame(sdevs_2022.values, index=[blank_index], columns=['SD'])

    for i in range(0,19):
        feature_labels[i] = key_features.columns[i]
    fl = pd.DataFrame(feature_labels, index=blank_index, columns=['Feature'])

    title_case('2020')
    display_side_by_side(fl, importances_list[0],importances_list[1], importances_list[2], importances_list[3], mean2020, sdev2020)
    
    title_case('2021')
    display_side_by_side(fl, importances_list[4],importances_list[5], importances_list[6], importances_list[7], mean2021, sdev2021)

    title_case('2022')
    display_side_by_side(fl, importances_list[8],importances_list[9], importances_list[10], importances_list[11], mean2022, sdev2022)

feature_importance()


					2020


,Feature
,Red
,Green
,Blue
,BI
,GLI
,NGRDI
,VARI
,BGI
,ExG
,ExR


					2021


,Feature
,Red
,Green
,Blue
,BI
,GLI
,NGRDI
,VARI
,BGI
,ExG
,ExR


					2022


,Feature
,Red
,Green
,Blue
,BI
,GLI
,NGRDI
,VARI
,BGI
,ExG
,ExR
